In [2]:
# 上下文管理器与 __enter__ 方法返回对象的区别

class LookingGlass:
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'

    def reverse_write(self, text):
        self.original_write(text[::-1])
    
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

with LookingGlass() as what:    # 这里的 what 是 __enter__ 的返回值
    print('Alice')
    print(what)

print(what)
print('Back to normal')

ecilA
YKCOWREBBAJ
JABBERWOCKY
Back to normal


In [3]:
# contextmanager 的使用

import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    sys.stdout.write = lambda text: original_write(text[::-1])
    msg = ''
    try:
        # 在当前上下文中抛出的异常，会通过 yield 语句抛出
        yield 'abcdefg'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

# 写完以后感觉这个用法跟 pytest.fixture 好像啊

with looking_glass() as what:
    print('ahhhhh')
    print(what)

print(what)

hhhhha
gfedcba
abcdefg
